In [ ]:
import re

str = "Hello, my name is Weiyu. I'm looking for a graduate school\n Thx!"
# str = str.lower()
print(str)

str_new = re.split('\s| , | .',str)
print(str_new)
str_new = [str for str in str_new if len(str)!=0]
print(str_new)
str_new = [re.sub("[^a-zA-Z0-9']", '',str) for str in str_new]
print(str_new)

In [ ]:
# keys = grouped_director_prof.groups.keys()

# from openpyxl import Workbook

# wb = Workbook()
# dest_filename = '/Users/weiyuchen/Downloads/datavis-project-2022-bcd-main/directors.xlsx'

# for key in keys:
#     sheet_name = key 
#     # print(type(sheet_name))
#     if(len(key)>31): sheet_name = sheet_name[:31]
#     wb.create_sheet(sheet_name)

# wb.save(filename = dest_filename)

# with pd.ExcelWriter('/Users/weiyuchen/Downloads/datavis-project-2022-bcd-main/directors.xlsx', mode = 'w') as writer:  
#     for key in keys:
#         sheet_name = key 
#         # print(type(sheet_name))
#         if(len(key)>31): sheet_name = sheet_name[:31]
#         temp = pd.DataFrame(grouped_director_prof.get_group(key))
#         temp.to_excel(writer, sheet_name=sheet_name,float_format="%f")



In [ ]:
# import http.client
# import mimetypes
# import re
# def getImgLink(id = 'tt0222012'):
#     conn = http.client.HTTPSConnection("imdb-api.com", 443)
#     payload = ''
#     headers = {}
#     conn.request("GET", "/en/API/Title/k_pz2zb0vf/"+id, payload, headers)
#     res = conn.getresponse()
#     data = res.read()
#     data = data.decode("utf-8")
#     # print(type(data))
#     print(data)
#     first_result = re.findall('"image":"https?://[^\s]*jpg', data)[0]
#     print(first_result[9:])
#     return first_result[9:]
# getImgLink()

In [ ]:
page_py=wikipedia.page('Wheels(film)')
print("Page - Images: %s" % page_py.images[0])
# print("Page - Title: %s" % wikipedia.summary("Wheels(film)"))
# print("Page - Summary: %s" % wikipedia.summary('Wheels(film)', sentences =1))
print(page_py)

Page - Images: https://upload.wikimedia.org/wikipedia/commons/d/d0/2011_Greater_Los_Angeles_Auto_Show_IMG_4306_%286870793840%29.jpg
<WikipediaPage 'Hot Wheels'>


In [ ]:
# output_world_link = output_world.copy()
# for ind in output_world_link.index:
#     output_world_link.at[ind, 0] = getImgLink(output_world_link.at[ind, 0])
#     output_world_link.at[ind, 1] = getImgLink(output_world_link.at[ind, 1])
# print(output_world_link)

In [ ]:
# import re

# str = "Hello, my name is Weiyu. I'm looking for a graduate school\n Thx!"
# # str = str.lower()
# print(str)

# str_new = re.split('\s| , | .',str)
# print(str_new)
# str_new = [str for str in str_new if len(str)!=0]
# print(str_new)
# str_new = [re.sub("[^a-zA-Z0-9']", '',str) for str in str_new]
# print(str_new)

Hello, my name is Weiyu. I'm looking for a graduate school
 Thx!
['Hello,', 'my', 'name', 'is', 'Weiyu.', "I'm", 'looking', 'for', 'a', 'graduate', 'school', '', 'Thx!']
['Hello,', 'my', 'name', 'is', 'Weiyu.', "I'm", 'looking', 'for', 'a', 'graduate', 'school', 'Thx!']
['Hello', 'my', 'name', 'is', 'Weiyu', "I'm", 'looking', 'for', 'a', 'graduate', 'school', 'Thx']


In [ ]:
from selenium import webdriver
import time
import os
import re
import requests
from PIL import Image
from io import BytesIO
from selenium.webdriver.common.by import By



class Crawler_google_images:
    # 初始化
    def __init__(self, keyword):
        self.url = 'https://www.bing.com/images/search?q='+keyword +' film poster'+'&tbm=isch'
        self.keyword = keyword

    # 获得Chrome驱动，并访问url
    def init_browser(self):
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-infobars")
        browser = webdriver.Chrome(chrome_options=chrome_options)
        # 访问url
        browser.get(self.url)
        # 最大化窗口，之后需要爬取窗口中所见的所有图片
        browser.maximize_window()
        return browser

    #下载图片
    def download_images(self, browser,round=2):
        picpath = './images/'
        # 路径不存在时创建一个
        if not os.path.exists(picpath): os.makedirs(picpath)

        count = 0 #图片序号
        pos = 0
        for i in range(round):
            pos += 500
            # 向下滑动
            js = 'var q=document.documentElement.scrollTop=%d'%pos
            browser.execute_script(js)
            time.sleep(1)
            # 找到图片
            # html = browser.page_source#也可以抓取当前页面的html文本，然后用beautifulsoup来抓取
            #直接通过tag_name来抓取是最简单的，比较方便

            img_elements = browser.find_elements(By.CLASS_NAME, 'iusc')
            # print(img_elements)
            for img_element in img_elements:
                m = img_element.get_attribute('m')
                # print(m)
                img_url = re.findall("murl\":\"https?://[^\s]*.jpg", m)
                # print(img_url[0][7:])
                img_url = img_url[0][7:]
                filename = picpath+'/'+ self.keyword + ".jpg"
                r = requests.get(img_url)
                img = Image.open(BytesIO(r.content))
                width, height = img.size
                print("width: ", width , ", height: ", height)
                if(height>width):
                    with open(filename, 'wb') as f:
                        f.write(r.content)
                    f.close()
                    count += 1
                    print('this is '+'img %d'%count)
                #防止反爬机制
                time.sleep(0.2)
                if(count>0): break
            if(count>0): break
    def run(self):
        browser = self.init_browser()
        self.download_images(browser,10)#可以修改爬取的页面数，基本10页是100多张图片
        browser.close()
        print("爬取完成")


def fetchImage(keyword = 'Your Name.'):
    craw = Crawler_google_images(keyword)
    craw.run()